## Computer Vision Project 2 Task 2 Back propagation
In this task, you will implement the following to train a MLP:
1. forward pass
2. backward pass
3. weights update

The MLP has an input layer, one hidden layer, and one output layer.

The input layer, the hidden layer, and the output layer has 784 nodes, 128 nodes, and 10 nodes, respectively.

You can use only the given sigmoid function as activation function.

You cannot use library functions except:
* torch.add
* torch.mul
* torch.transpose
* torch.mm

In [ ]:
import torch
from torchvision import transforms, datasets
from torch.autograd import Variable
import torch.nn.functional as F
import numpy as np

torch.manual_seed(77)

In [2]:
def sigmoid(x):
  return torch.div(torch.tensor(1.0), torch.add(torch.tensor(1.0), torch.exp(torch.negative(x))))

def sigmoid_prime(x):
  return torch.mul(sigmoid(x), torch.subtract(torch.tensor(1.0), sigmoid(x)))

In [3]:
train_MNIST = datasets.MNIST("MNIST_data/", train=True, transform=transforms.ToTensor(), download=True)
train_loader = torch.utils.data.DataLoader(dataset=train_MNIST,
                                          shuffle=True,
                                          drop_last=True)
dtype = torch.float32
D_in, H, D_out = 784, 128, 10

Refer to the following equation to implement forward pass:

$$ z_1 = W_1 x + b_1 $$
$$ a_1 = \sigma(z_1) $$
$$ z_2 = W_2 x + b_2 $$
$$ a_2 = \sigma(z_2) $$


In [ ]:
# A weight and a bias for input nodes
w1 = Variable(torch.randn(D_in, H, dtype=dtype, requires_grad=True)) * np.sqrt(1. / D_in)
b1 = Variable(torch.randn(1, H, dtype=dtype, requires_grad=True)) * np.sqrt(1. / D_in)

# A weight and a bias for hidden nodes
w2 = Variable(torch.randn(H, D_out, dtype=dtype, requires_grad=True)) * np.sqrt(1. / H)
b2 = Variable(torch.randn(1, D_out, dtype=dtype, requires_grad=True)) * np.sqrt(1. / H)

learning_rate = 0.1
for epoch in range(5): 
  corrects = 0
  for i, data in enumerate(train_loader):
    x, y = data
    x = x.reshape((1,-1))
    y_onehot = torch.zeros((1,10))
    y_onehot[0,y] += 1


    ############################################################################
    # TODO: Implement the forward pass for the two-layer net                   #
    #                                                                          #
    ############################################################################

    z1 = 
    a1 = 
    z2 = 
    a2 = 

    ############################################################################
    #                             END OF YOUR CODE                             #
    ############################################################################
    
    diff = a2 - y_onehot
    
    ############################################################################
    # TODO: Implement the backword pass for the two-layer net and update the   #
    # parameters                                                               #
    ############################################################################

    # backward pass
    d_z2 = 
    d_b2 = 
    d_w2 = 

    d_a1 = 
    d_z1 = 
    d_b1 = 
    d_w1 = 

    # weight update
    w1 -= 
    b1 -= 
    w2 -= 
    b2 -= 

    ############################################################################
    #                             END OF YOUR CODE                             #
    ############################################################################
    
    if torch.argmax(a2) == y:
      corrects += 1

    if i % 10000 == 0:
      print("Epoch {}: {}/{}".format(epoch+1, i, len(train_MNIST)))
      
  print("Epoch {}, Accuracy: {:.3f}".format(epoch+1, corrects/len(train_MNIST))) 

